In [141]:
import sqlite3
import pandas as pd
import json

pd.options.display.max_rows = 999

In [142]:
# res = con.execute("SELECT name FROM sqlite_master WHERE type='table';")
# for name in res:
#     print(name[0])

In [143]:
def read_pars_from_db(db):
    conn = sqlite3.connect(db)
    df_tr = pd.read_sql_query("select * from trial_values;", conn).dropna()

    pars = pd.read_sql_query("select * from trial_params;", conn).dropna()
    pars['d_name'] = pars.apply(lambda row: json.loads(row['distribution_json'])['name'] ,axis=1)
    pars['attributes'] = pars.apply(lambda row: json.loads(row['distribution_json'])['attributes'] ,axis=1)
    cat_dist = pars[pars.d_name == 'CategoricalDistribution']
    cat_dist = cat_dist.dropna(subset = ['param_value'])
    cat_dist['param_value'] = cat_dist.param_value.astype(int)
    cat_dist['val'] = cat_dist['param_value']

    pars.loc[pars.d_name == 'CategoricalDistribution','param_value'] \
            = cat_dist.apply(lambda row: row.attributes['choices'][row.val], axis=1)
    pars = pars[['trial_id','param_name','param_value']]
    
    df_tr = df_tr.set_index('trial_id').join(pars.set_index('trial_id')).drop(columns=['objective'])
#     return df_tr.sort_values(by = ['value'], ascending = False)
#     print(df_tr)
    
    table = df_tr.pivot_table(index=['trial_value_id','value'], columns='param_name', values='param_value',
                fill_value=0,aggfunc=lambda x: x).reset_index()
    return table.sort_values(by = ['value'], ascending = False)

In [144]:
# con = sqlite3.connect('gen3.db')
ddf = read_pars_from_db('gen7.db')

In [145]:
pd.options.display.max_rows = 1001
ddf

param_name,trial_value_id,value,add_trans,agg_type,conv_type,fin_type_c,fin_type_e,lb_smth,lr,nd,nlayers_gnn,node_agg_type,weight_decay
48,49,1.223095,False,sum_gate,sage,diff_mlp,cos,True,0.005636,256,3.0,max,0.000954
77,78,1.220348,False,sum_gate,sage,diff_mlp,mm_sig,True,0.003137,256,3.0,max,0.000657
68,69,1.220091,False,sum_gate,sage,diff_mlp,cos,True,0.006149,256,3.0,max,0.001348
63,64,1.217692,False,sum_gate,sage,diff_mlp,cos,True,0.005813,256,3.0,max,0.001121
72,73,1.210898,False,sum_gate,sage,diff_mlp,cos,True,0.005566,256,3.0,max,0.000656
56,57,1.209072,False,sum_gate,sage,diff_mlp,stack,True,0.004919,256,3.0,max,0.000426
74,75,1.208498,False,sum_gate,sage,diff_mlp,cos,True,0.005638,256,3.0,max,0.001064
64,65,1.203775,False,sum_gate,sage,diff_mlp,cos,True,0.005714,256,3.0,max,0.000503
43,44,1.198150,False,sum_gate,sage,diff_mlp,cos,True,0.005508,256,3.0,max,0.000952
44,45,1.196943,False,sum_gate,sage,diff_mlp,cos,True,0.005399,256,3.0,max,0.000862


In [138]:
m_val = ddf[:70].groupby(['agg_type', 'conv_type', 'fin_type_c', 'fin_type_e', 'node_agg_type'])['value'].mean().reset_index()
m_val.sort_values(by = ['value'], ascending = False)

,agg_type,conv_type,fin_type_c,fin_type_e,node_agg_type,value
1,mean_gate,resgate,diff_mlp,diff_mlp,max,1.796863
6,softmax,resgate,diff_mlp,diff_mlp,max,1.796027
5,mean_gate,sg,diff_mlp,diff_mlp,max,1.794955
2,mean_gate,resgate,diff_mlp,diff_mlp,mean,1.793746
3,mean_gate,sage,diff_mlp,diff_mlp,max,1.792866
0,mean,resgate,diff_mlp,diff_mlp,max,1.790932
4,mean_gate,sage,diff_mlp,diff_mlp,mean,1.789955


In [124]:
ddf.shape

(378, 7)

In [85]:
ddf.shape

(655, 10)

In [107]:
ddf[:20].weight_decay.mean()

0.0065587224701723346

In [118]:
ddf[:20].lr.mean()

0.004583965687068546

In [109]:
ddf.columns

Index(['add_trans', 'agg_type', 'conv_type', 'fin_type_c', 'fin_type_e', 'lr', 'nd', 'nlayers_gnn', 'node_agg_type', 'weight_decay'], dtype='object', name='param_name')

In [117]:
ddf.groupby('agg_type')['value'].mean()

agg_type
gate       1.485815
mean       1.691945
softmax    1.696963
Name: value, dtype: float64